In [55]:
import os
import json
from glob import glob

import numpy as np

In [12]:
path = '/home/datasets/COVID-CTset_visual'
trainfile = '/home/comp/18481086/code/hyperbox/hyperbox_app/covid19/datasets/covid_ctset/train_balance.json'
testfile = '/home/comp/18481086/code/hyperbox/hyperbox_app/covid19/datasets/covid_ctset/test_balance.json'

In [7]:
len(os.listdir('/home/datasets/COVID-CTset_visual/covid/patient1/SR_2'))

35

In [22]:
data = {}
info = {
    '#patients': 0,
    '#scans': 0
}
max_slices = -1
min_slices = 9999
for _cls in ['covid', 'normal']:
    data[_cls] = {}
    _path_patient = os.path.join(path, _cls)
    patients_num = len(os.listdir(_path_patient))
    print(f"{_cls} #patients={patients_num}")
    info['#patients'] += patients_num
    for _patient_id in os.listdir(_path_patient):
        data[_cls][_patient_id] = {}
        _path_scan = os.path.join(_path_patient, _patient_id)
        scans_num = len(os.listdir(_path_scan))
        info['#scans'] += scans_num
        for _scan_id in os.listdir(_path_scan):
            _path_slice = os.path.join(_path_scan, _scan_id)
            slices = os.listdir(_path_slice)
            data[_cls][_patient_id][_scan_id] = slices
            if len(slices)>max_slices:
                max_slices = len(slices)
                info['max_slices'] = max_slices
                info['max_slices_path'] = _path_slice
            if len(slices)<min_slices:
                min_slices = len(slices)
                info['min_slices'] = min_slices
                info['min_slices_path'] = _path_slice
print(info)
            


covid #patients=95
normal #patients=282
{'#patients': 377, '#scans': 1117, 'max_slices': 406, 'max_slices_path': '/home/datasets/COVID-CTset_visual/covid/patient78/SR_4', 'min_slices': 8, 'min_slices_path': '/home/datasets/COVID-CTset_visual/normal/patient222/SR_3'}


In [27]:
patients_num = 0
scans_num = 0
for _cls in train:
    patients_num += len(train[_cls])
    for _patient_id in train[_cls]:
        scans_num += len(train[_cls][_patient_id])
print('train', patients_num, scans_num)
patients_num = 0
scans_num = 0
for _cls in test:
    patients_num += len(test[_cls])
    for _patient_id in test[_cls]:
        scans_num += len(test[_cls][_patient_id])
print('test', patients_num, scans_num)
patients_num = 0
scans_num = 0
for _cls in data:
    patients_num += len(data[_cls])
    for _patient_id in data[_cls]:
        scans_num += len(data[_cls][_patient_id])
print('data', patients_num, scans_num)

train 280 402
test 97 124
data 377 1117


In [29]:
1117-402-124

591

In [28]:
def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)

def save_json(file, data):
    with open(file, 'w') as f:
        json.dump(data, f, indent=4)

In [32]:
save_json('/home/comp/18481086/code/hyperbox/hyperbox_app/covid19/datasets/iran/all_data.json', data)

In [14]:
train = load_json(trainfile)
test = load_json(testfile)

In [85]:
errors = {
    'cls_error': [],
    'slice_num_error_train': [],
    'slice_num_error_test': [],
    'scan_not_in_train': [],
    'scan_not_in_test': []
}
for _cls in data:
    for _patient_id in data[_cls]:
        in_train = _patient_id in train[_cls]
        in_test = _path_patient in test[_cls]
        if in_train and in_test:
            print(f'{_cls}_{patient_id} in both train&test')
            errors['cls_error'].append([_cls, _patient_id])
            continue
        for _scan_id in data[_cls][_patient_id]:
            num_data = len(data[_cls][_patient_id][_scan_id])
            if in_train:
                if _scan_id not in train[_cls][_patient_id]:
                    # print(f'{_scan_id} not in train')
                    errors['scan_not_in_train'].append([_cls, _patient_id, _scan_id])
                else:
                    num_train = len(train[_cls][_patient_id][_scan_id])
                    if num_train != num_data:
                        # print('train slice num not match:',_cls, _patient_id, _scan_id, num_data, num_train)
                        errors['slice_num_error_train'].append([_cls, _patient_id, _scan_id, num_data, num_train])
            if in_test:
                if _scan_id not in test[_cls][_patient_id]:
                    # print(f'{_scan_id} not in test')
                    errors['scan_not_in_test'].append([_cls, _patient_id, _scan_id])
                else:
                    num_test = len(test[_cls][_patient_id][_scan_id])
                    if len(test[_cls][_patient_id][_scan_id]) != num_data:
                        # print('test slice num not match:',_cls, _patient_id, _scan_id, num_data, num_test)
                        errors['slice_num_error_test'].append([_cls, _patient_id, _scan_id, num_data, num_test])
                    

In [86]:
len(errors['scan_not_in_train']), len(errors['scan_not_in_test'])

(0, 0)

In [87]:
len(errors['slice_num_error_train']), len(errors['slice_num_error_test']),len(errors['cls_error'])

(0, 0, 0)

In [88]:
scan_not_in_train = np.array(errors['scan_not_in_train'])
scan_not_in_train_list = []
for [c, p, s] in scan_not_in_train:
    scan_not_in_train_list.append(f"{c}+{p}+{s}")
len(scan_not_in_train_list), scan_not_in_train_list[:5]

(0, [])

In [89]:
errors['slice_num_error_train']
slice_num_error_train = np.array(errors['slice_num_error_train'])
slice_num_error_train_list = []
for [c, p, s, _, _] in slice_num_error_train:
    slice_num_error_train_list.append(f"{c}+{p}+{s}")
len(slice_num_error_train_list), slice_num_error_train_list[:5]

(0, [])

In [90]:
for name in slice_num_error_train_list:
    assert name not in scan_not_in_train_list, '1'
    c, p, s = name.split('+')
    assert train.get(c, False).get(p, False).get(s, False)

In [91]:
# correct slice num
for name in slice_num_error_train_list:
    c, p, s = name.split('+')
    train[c][p][s] = data[c][p][s]

In [92]:
# correct non-exist slice
for name in scan_not_in_train_list:
    c, p, s = name.split('+')
    train[c][p][s] = data[c][p][s]

In [83]:
a = {'covid': {'p1': {'s1': [2]}}}
a = {**a, **{'covid': {'p1': {'s2': [2], 's23': [43]}}}}
a

{'covid': {'p1': {'s2': [2], 's23': [43]}}}

In [93]:
save_json('/home/comp/18481086/code/hyperbox/hyperbox_app/covid19/datasets/iran/train.json', train)
save_json('/home/comp/18481086/code/hyperbox/hyperbox_app/covid19/datasets/iran/test.json', test)

In [94]:
train_old = load_json(trainfile)
test_old = load_json(testfile)

In [95]:
train_new = load_json(trainfile.replace('covid_ctset', 'iran').replace('_balance', ''))
test_new = load_json(testfile.replace('covid_ctset', 'iran').replace('_balance', ''))

In [96]:
for c in test_new:
    for p in test_new[c]:
        for s in test_new[c][p]:
            assert len(test_new[c][p][s])==len(test_old[c][p][s])

In [105]:
diff = {}
for c in train_new:
    for p in train_new[c]:
        for s in train_new[c][p]:
            if s not in train_old[c][p]:
                diff[f"{c}+{p}+{s}"] = 'not exist in old train'
            elif len(train_new[c][p][s])!=len(train_old[c][p][s]):
                diff[f"{c}+{p}+{s}"] = [len(train_new[c][p][s]), len(train_old[c][p][s])]
diff

{'covid+patient1+SR_2': [35, 19],
 'covid+patient1+SR_3': [35, 13],
 'covid+patient1+SR_4': [192, 36],
 'covid+patient20+SR_2': [47, 4],
 'covid+patient20+SR_3': [35, 2],
 'covid+patient20+SR_4': [35, 4],
 'covid+patient21+SR_2': [46, 8],
 'covid+patient21+SR_3': 'not exist in old train',
 'covid+patient21+SR_4': 'not exist in old train',
 'covid+patient22+SR_2': [55, 7],
 'covid+patient22+SR_3': [59, 4],
 'covid+patient22+SR_4': [370, 3],
 'covid+patient24+SR_2': [43, 5],
 'covid+patient24+SR_3': [35, 5],
 'covid+patient24+SR_4': [35, 9],
 'covid+patient26+SR_2': [35, 1],
 'covid+patient26+SR_3': [35, 7],
 'covid+patient26+SR_4': [159, 4],
 'covid+patient28+SR_2': [48, 3],
 'covid+patient28+SR_3': [35, 1],
 'covid+patient28+SR_4': 'not exist in old train',
 'covid+patient29+SR_2': [47, 3],
 'covid+patient29+SR_3': [35, 2],
 'covid+patient29+SR_4': [35, 1],
 'covid+patient30+SR_2': [53, 3],
 'covid+patient30+SR_3': [35, 2],
 'covid+patient30+SR_4': [35, 1],
 'covid+patient31+SR_2': [63

In [106]:
a=b=0
for key, value in diff.items():
    if isinstance(value, list):
        a += 1
    else:
        b += 1
a,b

(401, 429)

In [111]:
def statistics(data):
    info = {}
    for c in data:
        info[f'{c}_#patient'] = len(data[c])
        info[f'{c}_#scan'] = 0
        for p in data[c]:
            info[f'{c}_#scan'] += len(data[c][p])
    print(info)

In [110]:
statistics(train_new), statistics(test_new)

{'covid_#patient': 80, 'covid_#scan': 236, 'normal_#patient': 200, 'normal_#scan': 595}
{'covid_#patient': 15, 'covid_#scan': 42, 'normal_#patient': 82, 'normal_#scan': 82}


(None, None)

In [112]:
a = {'CT-0_#patient': 178, 'CT-0_#scan': 178, 'CT-1_#patient': 479, 'CT-1_#scan': 479, 'CT-2_#patient': 88, 'CT-2_#scan': 88, 'CT-3_#patient': 32, 'CT-3_#scan': 32, 'CT-4_#patient': 2, 'CT-4_#scan': 2}
a

{'CT-0_#patient': 178,
 'CT-0_#scan': 178,
 'CT-1_#patient': 479,
 'CT-1_#scan': 479,
 'CT-2_#patient': 88,
 'CT-2_#scan': 88,
 'CT-3_#patient': 32,
 'CT-3_#scan': 32,
 'CT-4_#patient': 2,
 'CT-4_#scan': 2}

In [113]:
479+88+32+2

601